In [24]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from pymongo import MongoClient
import re
import time
import os

In [25]:
# Настройка WebDriver
options = webdriver.FirefoxOptions()
driver = webdriver.Firefox(options=options)

# Подключение к MongoDB
client = MongoClient("mongodb://localhost:27017/PSU")
db = client["JuralDetails"]
collection = db["emails"]

In [26]:
# Переход на страницу почты
driver.get("https://mail.psu.ru/")

# загрузка страницы
time.sleep(5)

email_input = driver.find_element(By.NAME, "Username")
email_input.send_keys("vaechh")

password_input = driver.find_element(By.NAME, "Password")
password_input.send_keys("JohnVae322")

login_button = WebDriverWait(driver, 3).until(
    EC.element_to_be_clickable((By.XPATH, "//input[@type='submit' and @name='login' and @value='Войти']"))
)
login_button.click()

time.sleep(5)  # Подождите, чтобы завершился вход

In [32]:
for i in range(5):
    print(f"Открываем письмо {i + 1}")
    # получаем список всех писем и клюкаем по i-тому 
    emails = driver.find_elements(By.XPATH, "//tr[@class='borderbottom']")
    email = emails[i]
    td_with_second_link = email.find_element(By.XPATH, ".//td[position() = last()-2]//a")
    td_with_second_link.click()
    
    time.sleep(5)
    
    # Получаем заголовок письма
    subject_text = driver.find_element(By.CSS_SELECTOR, "h2.msgSubject.padRight").text
    print(subject_text)
    
    # Получаем метаданные (от кого, дата, кому)
    msg_headers = driver.find_elements(By.CSS_SELECTOR, "div.msgHeaders p.msgHeader")
    
    # Регулярные выражения для извлечения информации
    from_pattern = r"От Кого:\s*(.*)"
    date_pattern = r"Дата:\s*(.*)"
    
    # Извлекаем данные с помощью регулярных выражений
    from_match = re.search(from_pattern, msg_headers[0].text)
    date_match = re.search(date_pattern, msg_headers[1].text)
    
    # Если регулярное выражение находит совпадение, извлекаем данные
    from_email = from_match.group(1) if from_match else None
    date_sent = date_match.group(1) if date_match else None

    try:
        # Найдем iframe, в котором содержится письмо
        iframe = driver.find_element(By.CSS_SELECTOR, "div.msgBody iframe")
    
        driver.switch_to.frame(iframe)
        iframe_html = driver.page_source
        
        email_data = {
            "subject": subject_text,
            "from": from_email,
            "date": date_sent,
            "html_content": iframe_html  # Сохраняем HTML как строку
        }
        
        collection.insert_one(email_data)

    except Exception as e:
        print(f"Ошибка при извлечении содержимого письма: {e}")

    finally:
        driver.switch_to.default_content()

    driver.back()
    
    time.sleep(5)

Открываем письмо 1
Ivan, people are recommending your work 
Открываем письмо 2
Ivan, you have new reads 
Открываем письмо 3
Ivan, people are reading your work 
Открываем письмо 4
Ivan, you have new reads 
Открываем письмо 5
Congratulations Ivan, you achieved top stats in November 
